# Demonstrate ADES Execution for OGC Application Packages
## This notebook runs through some example API calls to the ADES (Application, Deployment Execution Service) component of the EODH Platform

In [1]:
!pip install urllib3

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import time
import urllib3
http = urllib3.PoolManager(cert_reqs='CERT_NONE')
urllib3.disable_warnings() ## temporary fix only!

In [3]:
## Define text colour for later output
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'

## Below are some example API requests you can make to the ADES component
Feel free to run these examples and change the inputs by specifying the application packages, process name and process inputs.
All outputs can be found in the S3 bucket [eodhp-ades](https://s3.console.aws.amazon.com/s3/buckets/eodhp-ades?region=eu-west-2&bucketType=general&tab=objects).

As an example we provide an EOEPCA-developed OGC Application Package to demonstrate the successful execution using the ADES deployment:
- [convert-url](https://github.com/EOEPCA/convert/blob/main/convert-url-app.cwl) - take an image specified by a URL and resize it by a given scale percentage

This application is specified by configuring the below variable

In [4]:
process_to_be_run = "convert_url"

In [5]:
# Update these variables as required to identify the running ades instance and specify workspace name
# If the workspace does not yet exect, it will be created by the ades automatically
ades_endpoint = "ades.dev.eodhp.eco-ke-staging.com"
user = "tom"

# Automated configuration of CWL script location, process name and inputs
if process_to_be_run == "convert_url":
    process_name = "convert-url"
    cwl_location = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-url-app.cwl"
    inputs_dict = {"inputs": {
                    "fn": "resize",
                    "stac":  "https://raw.githubusercontent.com/EOEPCA/convert/main/stac/eoepca-logo.json",
                    "size": "50%"
                    }
                  }

### List processes

In [7]:
url = f"http://{ades_endpoint}/{user}/ogc-api/processes"
headers = {"Accept": "application/json"}

response = http.request('GET', url, headers=headers)
json.loads(response.data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Deploy processes

In [ ]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes"
headers = {"Accept": "application/json", "Content-Type": "application/json"}
params = {"executionUnit": {
            "href": f"{cwl_location}",
            "type": "application/cwl"
            }
         }
response = http.request('POST', url, headers=headers, body=json.dumps(params))
deployStatus = response.headers['Location']
json.loads(response.data)

### Get deploy status

In [ ]:
url = f"{deployStatus}"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

### Get process details

In [ ]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

### Execute process

In [ ]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}/execution"
headers = {"Accept": "application/json", "Content-Type": "application/json", "Prefer": "respond-async"}
params = {**inputs_dict,
          "response":"raw"
          }
response = http.request('POST', url, headers=headers, body=json.dumps(params))
executeStatus = response.headers['Location']
json.loads(response.data)

### Get execute status
See the following section to continually poll this function instead to determine once complete

In [ ]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

### Get execute status (continuous polling)
Run this cell to keep polling the ExecuteStatus endpoint to determine when the process has finished running and also see it's final status: *SUCCESS* or *FAILED*

In [ ]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
data = json.loads(response.data)
status = data['status']
print("Status is " + bcolors.OKGREEN + status.upper() + bcolors.ENDC)
while status == "running":
    time.sleep(5)
    response = http.request('GET', url, headers=headers)
    print("Status is " + bcolors.OKGREEN + status.upper() + bcolors.ENDC)

if status == "successful":
    print(bcolors.OKGREEN + "SUCCESS" + bcolors.ENDC)

if status == "failed":
    print(bcolors.WARNING + "FAILED" + bcolors.ENDC)

### Get processing results

In [ ]:
## Note, this will return a 500 response when no output is produced, e.g. when running do-nothing
url = f"{executeStatus}/results"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
data = json.loads(response.data)
data_location = data['StacCatalogUri'] if 'StacCatalogUri' in data else "No location provided"

print(f"Data location: {data_location}")

### List jobs

In [ ]:
url = f"https://{ades_endpoint}/{user}/ogc-api/jobs"
headers = {"Accept": "application/json"}
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

### Undeploy/Delete process

In [ ]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}"
headers = {"Accept": "application/json"}
params = {}
response = http.request('DELETE', url, headers=headers)
response.status